In [ ]:
import torch
import torch.nn as nn
import timm

from torch.utils.data import DataLoader

from torchvision.datasets import ImageFolder
from torchvision.transforms import (
    Compose,
    Resize,
    ToTensor,
)
import torchvision.models as models

import numpy as np
import os
from torchvision.models import ResNet101_Weights, MobileNet_V3_Large_Weights, EfficientNet_V2_S_Weights

print(f'Torch-Version {torch.__version__}')
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f'DEVICE: {DEVICE}')

#### **SSL base models feature extraction**

Place hooks to the SSL base models and extract features from training and testing data.

In [ ]:
class ResNetFeatureExtractor(nn.Module):
    def __init__(self, pretrained=True):
        super(ResNetFeatureExtractor, self).__init__()
        
        # Load the ResNet-101 model with pretrained weights
        resnet101 = models.resnet101(weights=ResNet101_Weights.DEFAULT )
        
        # Exclude average pooling and classification layers
        self.conv_layers = torch.nn.Sequential(*list(resnet101.children())[:-2])
        self.maxpool = nn.AdaptiveMaxPool2d((1, 1))  # Add AdaptiveMaxPool2d
        
    def forward(self, x):
        features = self.conv_layers(x)
        features = self.maxpool(features)  # Apply AdaptiveMaxPool2d
        features = torch.flatten(features, 1)  # Flatten features
        return features

In [ ]:
class MobileNetFeatureExtractor(nn.Module):
    def __init__(self, pretrained=True):
        super(MobileNetFeatureExtractor, self).__init__()
        
        # Load the MobileNet model with pretrained weights
        Mobile_l = models.mobilenet_v3_large(weights=MobileNet_V3_Large_Weights.DEFAULT )
        
        # Exclude average pooling and classification layers
        self.conv_layers = torch.nn.Sequential(*list(Mobile_l.children())[:-2])
        self.maxpool = nn.AdaptiveMaxPool2d((1, 1))  # Add AdaptiveMaxPool2d
        
    def forward(self, x):
        features = self.conv_layers(x)
        features = self.maxpool(features)  # Apply AdaptiveMaxPool2d
        features = torch.flatten(features, 1)  # Flatten features
        return features

In [ ]:
class EfficientNetFeatureExtractor(nn.Module):
    def __init__(self, pretrained=True):
        super(EfficientNetFeatureExtractor, self).__init__()
        
        # Load the EfficientNet model with pretrained weights
        Efficient = models.efficientnet_v2_s(weights=EfficientNet_V2_S_Weights.DEFAULT )
        
        # Exclude average pooling and classification layers
        self.conv_layers = torch.nn.Sequential(*list(Efficient.children())[:-2])
        self.maxpool = nn.AdaptiveMaxPool2d((1, 1))  # Add AdaptiveMaxPool2d
        
    def forward(self, x):
        features = self.conv_layers(x)
        features = self.maxpool(features)  # Apply AdaptiveAvgPool2d
        features = torch.flatten(features, 1)  # Flatten features
        return features

In [ ]:
class Identity(nn.Module):
    def __init__(self):
        super(Identity, self).__init__()
    def forward(self, x):
        return x

class SimCLR(nn.Module):
    def __init__(self, linear_eval=False):
        super().__init__()
        self.linear_eval = linear_eval
        feature_extractor = ResNetFeatureExtractor()
        self.encoder = feature_extractor
        self.projection = nn.Sequential(
            nn.Linear(2048, 128),
            nn.ReLU(),
            nn.Linear(128,128)
        )
    def forward(self, x):
        if not self.linear_eval:
            x = torch.cat(x, dim=0)
    
        encoding = self.encoder(x)
        projection = self.projection(encoding) 
        return projection

In [ ]:
# Define the input data size
resize = Resize((224, 224))
ttensor = ToTensor()

custom_transform = Compose([
    resize,
    ttensor,
])

# Set the input data path
sample_train_case = ImageFolder(
    root="target_dataset/train_images_directory/",
    transform=custom_transform
)

sample_test_case = ImageFolder(
    root="target_dataset/test_images_directory/",
    transform=custom_transform
)

BATCH_SIZE = 64

train_dataset = sample_train_case
test_dataset = sample_test_case

# Building the data loader
train_dl = DataLoader(
    train_dataset,
    batch_size=BATCH_SIZE,
    num_workers=os.cpu_count(),
    pin_memory=True,
)

test_dl = DataLoader(
    test_dataset,
    batch_size=BATCH_SIZE,
    num_workers=os.cpu_count(),
    pin_memory=True,
)

In [ ]:
class LinearEvaluation(nn.Module):
    def __init__(self, model, nu_classes):
        super().__init__()
        simclr = model
        simclr.linear_eval = True
        simclr.projection = Identity()
        self.simclr = simclr
        for param in self.simclr.parameters():
            param.requires_grad = False
        self.linear = nn.Sequential(nn.Linear(2048, 128), nn.ReLU())
        self.linear2 = nn.Sequential(nn.Linear(128, nu_classes), nn.Softmax(dim=-1))
    def forward(self, x):
        encoding = self.simclr(x)
        encoding = self.linear(encoding) 
        pred = self.linear2(encoding)
        return pred

In [ ]:
# Load trained base models
new_effici_model = torch.load('saved_model/pre-trained_model.pth')
new_resnet_model = torch.load('saved_model/pre-trained_model.pth')
new_mobile_model = torch.load('saved_model/pre-trained_model.pth')

In [ ]:
# placeholder for batch features
features = {}

def get_features(name):
    def hook(model, input, output):
        features[name] = output.detach()

    return hook

##### REGISTER HOOK

new_resnet_model.linear.register_forward_hook(get_features("resnet_feats"))
new_mobile_model.linear.register_forward_hook(get_features("mobile_feats"))
new_effici_model.linear.register_forward_hook(get_features("effici_feats"))

In [ ]:
ResNet_FEATS = []
MobileNet_FEATS = []
EfficiNet_FEATS = []
Train_Label = []

with torch.no_grad():
        for i, element in enumerate(train_dl):
                image, label = element
                image = image.to(DEVICE)
                label = label.to(DEVICE)
                resnet_pred = new_resnet_model(image)
                mobile_pred = new_mobile_model(image)
                effici_pred = new_effici_model(image)

                Train_Label.append(label.cpu().numpy())
                ResNet_FEATS.append(features["resnet_feats"].cpu().numpy())
                MobileNet_FEATS.append(features["mobile_feats"].cpu().numpy())
                EfficiNet_FEATS.append(features["effici_feats"].cpu().numpy())

In [ ]:
ResNet_Test_FEATS = []
MobileNet_Test_FEATS = []
EfficiNet_Test_FEATS = []
Test_Label = []

with torch.no_grad():
        for i, element in enumerate(test_dl):
                image, label = element
                image = image.to(DEVICE)
                label = label.to(DEVICE)
                resnet_pred = new_resnet_model(image)
                mobile_pred = new_mobile_model(image)
                effici_pred = new_effici_model(image)

                Test_Label.append(label.cpu().numpy())
                ResNet_Test_FEATS.append(features["resnet_feats"].cpu().numpy())
                MobileNet_Test_FEATS.append(features["mobile_feats"].cpu().numpy())
                EfficiNet_Test_FEATS.append(features["effici_feats"].cpu().numpy())

In [ ]:
ResNet_FEATS_Con = []
MobileNet_FEATS_Con = []
EfficiNet_FEATS_Con = []

ResNet_FEATS_Con = np.concatenate(ResNet_FEATS)
MobileNet_FEATS_Con = np.concatenate(MobileNet_FEATS)
EfficiNet_FEATS_Con = np.concatenate(EfficiNet_FEATS)

print("- feats shape:", ResNet_FEATS_Con.shape)
print("- feats shape:", MobileNet_FEATS_Con.shape)
print("- feats shape:", EfficiNet_FEATS_Con.shape)

In [ ]:
ResNet_Test_FEATS_Con = []
MobileNet_Test_FEATS_Con = []
EfficiNet_Test_FEATS_Con = []

ResNet_Test_FEATS_Con = np.concatenate(ResNet_Test_FEATS)
MobileNet_Test_FEATS_Con = np.concatenate(MobileNet_Test_FEATS)
EfficiNet_Test_FEATS_Con = np.concatenate(EfficiNet_Test_FEATS)

print("- feats shape:", ResNet_Test_FEATS_Con.shape)
print("- feats shape:", MobileNet_Test_FEATS_Con.shape)
print("- feats shape:", EfficiNet_Test_FEATS_Con.shape)

In [ ]:
Train_Label = np.concatenate(Train_Label, axis=0)

print("Labels shape:", Train_Label.shape)

Test_Label = np.concatenate(Test_Label, axis=0)

print("Labels shape:", Test_Label.shape)

#### **TL base models feature extraction**

Place hooks to the base models and extract features from training and testing data.

In [ ]:
import os
import cv2
import numpy as np
import pandas as pd

from tensorflow.keras.utils import to_categorical
from keras.models import load_model
from sklearn.metrics import precision_recall_fscore_support

In [ ]:
# Create training dataset direction
dataset = "sample"
if dataset == "sample":
    dataset_dir = "origin_dataset/train_images_directory" #Need to adjust the data path as the training data set

# Read data files and class labels 
def get_dataCategories(dataset_dir):
    import glob

    categories = []
    if dataset == "sample":   #Add more sample functions to allow more input datasets for choosing
        for folder_name in os.listdir(dataset_dir):
            if os.path.isdir(os.path.join(dataset_dir, folder_name)):
                nbr_files = len(
                    glob.glob(os.path.join(dataset_dir, folder_name) + "/*.jpg") #Need to adjust the sample image suffix format to suit the input data
                )
                categories.append(np.array([folder_name, nbr_files]))  
    elif dataset == "sample2":
        for folder_name in os.listdir(dataset_dir):
            if os.path.isdir(os.path.join(dataset_dir, folder_name)):
                nbr_files = len(
                    glob.glob(os.path.join(dataset_dir, folder_name) + "/*.jpg")
                )
                categories.append(np.array([folder_name, nbr_files]))


    categories.sort(key=lambda a: a[0])
    cat = np.array(categories)

    return list(cat[:, 0]), list(cat[:, 1])


categories, nbr_files = get_dataCategories(dataset_dir)

# Create DataFrame
df = pd.DataFrame({"categorie": categories, "numbre of files": nbr_files})
print("number of categories: ", len(categories))
df  # Provide a data frame of your input data

In [ ]:
# Create data tensor
def create_dataset(datadir, categories, img_wid, img_high):
    
    X, y = [], []
    for category in categories:
        path = os.path.join(datadir, category)
        class_num = categories.index(category)
        for img in os.listdir(path):
            try:
                img_array = cv2.imread(os.path.join(path, img))
                ima_resize_rgb = cv2.resize(img_array, (img_wid, img_high))

                X.append(ima_resize_rgb) 
                y.append(class_num)

            except Exception as e:
                pass

    y = np.array(y)
    X = np.array(X).reshape(y.shape[0], img_wid, img_high, 3) # Resize the data to match the input size requirement, following the direction: sample number, image width, image height, colour channel
    return X, y


img_wid, img_high = 224, 224 # Define the data size

X, y = create_dataset(dataset_dir, categories, img_wid, img_high) 

print(f"X: {X.shape}")
print(f"y: {y.shape}")


In [ ]:
# Create testing dataset direction
if dataset == "sample":
    test_dataset_dir = "origin_dataset/test_images_directory"
elif dataset == "sample2":
    test_dataset_dir = "origin_dataset/test_images_directory"


# Converting to scaler format
Y = np.reshape(y, (len(y), 1))

x_train = X
y_train = Y

x_test, y_t = create_dataset(test_dataset_dir, categories, img_wid, img_high)

# One Hot Encoding
y_train = to_categorical(y_train)
y_test = to_categorical(y_t)

# Verifying the dimension after one hot encoding
print(f"x_train:{x_train.shape},  y_train:{y_train.shape}")
print(f"x_test:{x_test.shape},  y_test:{y_test.shape}")

In [ ]:
#Load pre-trained base models
mobilenet_model = load_model("saved_model/finetuned_mobilenet.keras")
incepresnet_model = load_model("saved_model/finetuned_incepresnet.keras")
xception_model = load_model("saved_model/finetuned_xception.keras")

In [ ]:
from keras.models import Model

mobilenet_model = Model(inputs=mobilenet_model.input, outputs=mobilenet_model.get_layer(index=-2).output)
incepresnet_model = Model(inputs=incepresnet_model.input, outputs=incepresnet_model.get_layer(index=-2).output)
xception_model = Model(inputs=xception_model.input, outputs=xception_model.get_layer(index=-2).output)

mobilenet_train_prediction = np.array(mobilenet_model.predict(x_train))
mobilenet_Xtrain = np.reshape(mobilenet_train_prediction, (mobilenet_train_prediction.shape[0], mobilenet_train_prediction.shape[1]))

mobilenet_test_prediction = np.array(mobilenet_model.predict(x_test))
mobilenet_Xtest = np.reshape(mobilenet_test_prediction, (mobilenet_test_prediction.shape[0], mobilenet_test_prediction.shape[1]))

print('\tMobilenet features training shape: ', mobilenet_Xtrain.shape)
print('\tMobilenet features testing shape: ', mobilenet_Xtest.shape)

incepresnet_train_prediction = np.array(incepresnet_model.predict(x_train))
incepresnet_Xtrain = np.reshape(incepresnet_train_prediction, (incepresnet_train_prediction.shape[0], incepresnet_train_prediction.shape[1]))

incepresnet_test_prediction = np.array(incepresnet_model.predict(x_test))
incepresnet_Xtest = np.reshape(incepresnet_test_prediction, (incepresnet_test_prediction.shape[0], incepresnet_test_prediction.shape[1]))

print('\tIncepresnet features training shape: ', incepresnet_Xtrain.shape)
print('\tIncepresnet features testing shape: ', incepresnet_Xtest.shape)

xception_train_prediction = np.array(xception_model.predict(x_train))
xception_Xtrain = np.reshape(xception_train_prediction, (xception_train_prediction.shape[0], xception_train_prediction.shape[1]))

xception_test_prediction = np.array(xception_model.predict(x_test))
xception_Xtest = np.reshape(xception_test_prediction, (xception_test_prediction.shape[0], xception_test_prediction.shape[1]))

print('\tXception features training shape: ', xception_Xtrain.shape)
print('\tXception features testing shape: ', xception_Xtest.shape)

#### **Feature fusion**

Concatenate extracted features from both pre-trained both side base models.

In [ ]:
FEATS = []
Test_FEATS = []

FEATS = np.concatenate((ResNet_FEATS_Con, MobileNet_FEATS_Con, EfficiNet_FEATS_Con, incepresnet_Xtrain, xception_Xtrain, mobilenet_Xtrain), axis=1)
print("- preds shape:", FEATS.shape)

Test_FEATS = np.concatenate((ResNet_Test_FEATS_Con, MobileNet_Test_FEATS_Con, EfficiNet_Test_FEATS_Con, incepresnet_Xtest, xception_Xtest, mobilenet_Xtest), axis=1)
print("- test preds shape:", Test_FEATS.shape)

#### **Feature selection**

Use feature selection methods to filter the concatenated features.

In [ ]:
from sklearn.decomposition import FastICA

# Initialise the ICA model
ica = FastICA(n_components=50) # Change the n_components to the number of features that want to keep
ica.fit(FEATS)

X_train_ica = ica.transform(FEATS)
X_test_ica = ica.transform(Test_FEATS)

X_train_ica.shape

#### **Ensemble learning**

Use selected features as the input data to train the higher-level ML classifier for ensemble learning, and use selected testing features to test the performance.

In [ ]:
from sklearn.svm import SVC, LinearSVC
from sklearn.metrics import accuracy_score

print("Classification with Linear SVM ...")
svm_model = LinearSVC(dual="auto", random_state=0, tol=1e-2)
svm_model.fit(X_train_ica, np.ravel(y, order='C'))
svm_result = svm_model.predict(X_test_ica)

x_pure_test, y_pure_test = create_dataset(test_dataset_dir, categories, img_wid, img_high)

acc = accuracy_score(svm_result, np.ravel(y_pure_test, order='C'))
precision, recall, f1_score, _ = precision_recall_fscore_support(svm_result, y_pure_test, average=None)

print("\tAccuracy Linear SVM: %0.4f" % acc)
for i in range(len(precision)):
    print(f"Class {i} - Precision: {precision[i]}, Recall: {recall[i]}, F1-score: {f1_score[i]}")

In [ ]:
from xgboost import XGBClassifier

# Train XGBoost model
xgb_model = XGBClassifier(booster = 'gbtree', eta = 0.9, max_depth = 10)
xgb_model.fit(X_train_ica, y)
xgb_result = xgb_model.predict(X_test_ica)

x_pure_test, y_pure_test = create_dataset(test_dataset_dir, categories, img_wid, img_high)

acc = accuracy_score(xgb_result, np.ravel(y_pure_test, order='C'))
precision, recall, f1_score, _ = precision_recall_fscore_support(xgb_result, y_pure_test, average=None)

print("\tAccuracy XGBoost: %0.4f" % acc)
for i in range(len(precision)):
    print(f"Class {i} - Precision: {precision[i]}, Recall: {recall[i]}, F1-score: {f1_score[i]}")

In [ ]:
from sklearn.ensemble import RandomForestClassifier

# Train RandomForest Classifier
rf_model = RandomForestClassifier(min_samples_split=3, max_features='log2')
rf_model.fit(X_train_ica, y)
rf_result = rf_model.predict(X_test_ica)

x_pure_test, y_pure_test = create_dataset(test_dataset_dir, categories, img_wid, img_high)

acc = accuracy_score(rf_result, np.ravel(y_pure_test, order='C'))
precision, recall, f1_score, _ = precision_recall_fscore_support(rf_result, y_pure_test, average=None)

print("\tAccuracy RandomForest: %0.4f" % acc)
for i in range(len(precision)):
    print(f"Class {i} - Precision: {precision[i]}, Recall: {recall[i]}, F1-score: {f1_score[i]}")

In [ ]:
from sklearn.naive_bayes import GaussianNB

# Train Naive Bayes Classifier
nb_model = GaussianNB()
nb_model.fit(X_train_ica, y)
nb_result = nb_model.predict(X_test_ica)

x_pure_test, y_pure_test = create_dataset(test_dataset_dir, categories, img_wid, img_high)

acc = accuracy_score(nb_result, np.ravel(y_pure_test, order='C'))
precision, recall, f1_score, _ = precision_recall_fscore_support(nb_result, y_pure_test, average=None)

print("\tAccuracy NaiveBayes: %0.4f" % acc)
for i in range(len(precision)):
    print(f"Class {i} - Precision: {precision[i]}, Recall: {recall[i]}, F1-score: {f1_score[i]}")

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score

# Train k-Nearest Neighbors Classifier
knn_model = KNeighborsClassifier(n_neighbors=3, weights = 'distance', p=1)
knn_model.fit(X_train_ica, y)
knn_result = knn_model.predict(X_test_ica)

x_pure_test, y_pure_test = create_dataset(test_dataset_dir, categories, img_wid, img_high)

acc = accuracy_score(knn_result, np.ravel(y_pure_test, order='C'))
precision, recall, f1_score, _ = precision_recall_fscore_support(knn_result, y_pure_test, average=None)

print("\tAccuracy Linear regression: %0.4f" % acc)
for i in range(len(precision)):
    print(f"Class {i} - Precision: {precision[i]}, Recall: {recall[i]}, F1-score: {f1_score[i]}")

In [ ]:
from sklearn.ensemble import VotingClassifier

# Combine predictions into a list
all_predictions = [nb_result, xgb_result, rf_result, knn_result, svm_result]

# Initialise a voting classifier using hard voting (majority voting)
voting_clf = VotingClassifier(estimators=[
    ('naive_bayes', nb_model),
    ('xgboost', xgb_model),
    ('rf', rf_model),
    ('knn', knn_model),
    ('svm', svm_model),
], voting='hard')

# Reshape predictions to ensure they have at most two dimensions
reshaped_predictions = [pred.reshape(-1, 1) for pred in all_predictions]

# Fit the voting classifier on the reshaped predictions
voting_clf.fit(np.hstack(reshaped_predictions), y_pure_test)

# Make predictions using the voting classifier
ensemble_predictions = voting_clf.predict(np.hstack(reshaped_predictions))

# Evaluate the ensemble model
ensemble_accuracy = accuracy_score(y_pure_test, ensemble_predictions)
ensemble_precision, ensemble_recall, ensemble_f1_score, _ = precision_recall_fscore_support(y_pure_test, ensemble_predictions, average=None)

print("Ensemble Accuracy:", ensemble_accuracy)
print("Ensemble recall:", ensemble_recall)
print("Ensemble precision:", ensemble_precision)
print("Ensemble f1:", ensemble_f1_score)

#### **Confusion matrix evaluation**

Evaluate the final predictions with ground truth label, and provide a confusion matrix.

In [ ]:
def cm_plt(ax, cm, classes, cmap, title, normalize, font_size= 16 ):
    im = ax.imshow(cm, interpolation="nearest", cmap=cmap)
    ax.figure.colorbar(im, ax=ax)
    # We want to show all ticks...
    ax.set(
        xticks=np.arange(cm.shape[1]),
        yticks=np.arange(cm.shape[0]),
        # ... and label them with the respective list entries
        xticklabels=classes,
        yticklabels=classes,
        title=title,
        ylabel="True label",
        xlabel="Predicted label",
    )

    # Rotate the tick labels and set their alignment.
    plt.setp(ax.get_xticklabels(), rotation=45, ha="right", rotation_mode="anchor", fontsize=font_size)
    plt.setp(ax.get_yticklabels(), fontsize=font_size)
    # Loop over data dimensions and create text annotations.
    fmt = ".2f" if normalize else "d"
    thresh = cm.max() / 2.0
    for i in range(cm.shape[0]):
        for j in range(cm.shape[1]):
            ax.text(
                j,
                i,
                format(cm[i, j], fmt),
                ha="center",
                va="center",
                color="white" if cm[i, j] > thresh else "black",
                fontsize=font_size
            )

    return ax

# Defining function for confusion matrix plot
def plt_confusion_mat(cm, classes, fig_size, cmap=plt.cm.Blues, font_size=16):
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=fig_size)
    ax1 = cm_plt(
        ax1,
        cm,
        classes,
        cmap,
        title="Confusion matrix, without normalization",
        normalize=False,
        font_size=font_size
    )

    cmn = cm.astype("float") / cm.sum(axis=1)[:, np.newaxis]
    ax2 = cm_plt(
        ax2,
        cmn,
        classes,
        cmap,
        title="Normalized confusion matrix",
        normalize=True,
        font_size=font_size
    )

In [ ]:
from sklearn.metrics import confusion_matrix
# Store class label in sequence of saving data 
Label = ['x','xx'] 
# Convert predicted_results and ground_truth_labels to NumPy arrays
predicted_results = np.array(ensemble_predictions)
ground_truth_labels = np.array(y_pure_test)

# Calculate the confusion matrix
cm = confusion_matrix(predicted_results, ground_truth_labels)

# Call the confusion matrix plotting functions
fig_size = (24, 7)  # Adjust the figure size as needed
plt_confusion_mat(cm, Label, fig_size)